In [1]:
import os
import pandas as pd

In [2]:
data_directory = "./FedTMD_fit_costs_per_round_no_selection/"  # Substitua pelo diretório correto onde estão os arquivos .csv


In [3]:
# Lista para armazenar os DataFrames carregados de cada arquivo .csv
data_frames = []

output_concat_csv = "./aggregated_communication_costs_results_without_selection.csv"

In [4]:
try:
    df_results = pd.read_csv(output_concat_csv)
except FileNotFoundError:
    df_results = pd.DataFrame(columns=['num_rounds','learning_rate', 'batch_size', 'local_epochs', 'accuracy', 'precision', 'recall', 'cost', 'bound'])

def calcular_f1_score(row):
    return (2 * ((row['precision'] * row['recall']) / (row['precision'] + row['recall'])))


# Loop pelas pastas no diretório
for folder in os.listdir(data_directory):
    folder_path = os.path.join(data_directory, folder)
    if os.path.isdir(folder_path):  # Verifica se é uma pasta
        # Extrair os parâmetros do nome da pasta
        folder_parts = folder.split("_")
        num_rounds = int(folder_parts[0])
        learning_rate = float(folder_parts[1])
        batch_size = int(folder_parts[2])
        local_epochs = int(folder_parts[3])
        
        # Localizar o arquivo aggregated_evaluation_metrics.csv dentro da pasta
        file_path = os.path.join(folder_path, "aggregated_fit_metrics.csv")

        
        if os.path.exists(file_path):
            # Carregar o arquivo .csv original em um DataFrame do pandas
            original_df = pd.read_csv(file_path)
            
            # Criar novo dataframe para as médias
            #df_results = pd.DataFrame(columns=['num_rounds','learning_rate', 'batch_size', 'local_epochs', 'loss', 'accuracy', 'precision', 'recall', 'cost', 'bound'])
            

            # Adicionar colunas de parâmetros ao DataFrame
            df_feature = pd.DataFrame(
                    {'num_rounds': [num_rounds] ,
                     'learning_rate': [learning_rate], 
                     'batch_size': [batch_size], 
                     'local_epochs': [local_epochs] ,
                     'cost': [int(original_df['cost'].sum())],
                     'bound': [int(original_df['bound'].iloc[0])],
                     'accuracy': [original_df['accuracy'].mean()] , 
                     'precision': [original_df["precision"].mean()]  , 
                     'recall': [original_df["recall"].mean()] 
                     }
                    )
            
            df_feature['f1_score'] = original_df.apply(calcular_f1_score, axis=1)
            df_feature['f1_score'] = df_feature['f1_score'].values[0]


            # Concatene o novo dataframe com o dataframe existente
            df_results = pd.concat([df_results, df_feature], ignore_index=True)

        else:
            print(f"Warning: File 'aggregated_fit_metrics.csv' not found in folder {folder}.")
    

    


C:\Users\windows\AppData\Local\Temp\ipykernel_8980\2193557738.py:7: RuntimeWarning: invalid value encountered in double_scalars
  return (2 * ((row['precision'] * row['recall']) / (row['precision'] + row['recall'])))


In [5]:
    # Concatenar todos os DataFrames em um único DataFrame
    #df_concat = pd.concat(data_frames, ignore_index=True)
    # Salvar o DataFrame concatenado como um arquivo CSV

    #df_concat.to_csv(output_concat_csv, index=False)
    #new_df.to_csv(output_concat_csv, index=False)
df_results.to_csv(output_concat_csv, index=False)
             
